In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [2]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1))

# define input and load data

In [3]:
product_urlpath = (
    "data/S1B_S6_SLC__1SDV_20211216T115438_20211216T115501_030050_03968A_4DCB.SAFE/"
)
measurement_group = "S6/VV"
dem_urlpath = "data/Chicago-4m-DEM.tif"
orbit_group = None
calibration_group = None
output_urlpath = "Chicago-4m-GTC-SLC.tif"
correct_radiometry = False
interp_method = "nearest"
multilook = None
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {"chunks": {}}
kwargs = {"chunks": 2048}

In [4]:
!ls -d {product_urlpath}
!ls -d {dem_urlpath}

data/S1B_S6_SLC__1SDV_20211216T115438_20211216T115501_030050_03968A_4DCB.SAFE/
data/Chicago-4m-DEM.tif


In [5]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

# scene

In [6]:
dem_raster

<xarray.DataArray 'band_data' (y: 6250, x: 6250)>
dask.array<getitem, shape=(6250, 6250), dtype=float32, chunksize=(6250, 6250), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05 4e+05
  * y            (y) float64 4.625e+06 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
    spatial_ref  int64 ...

In [7]:
# _ = dem_raster.plot()

In [8]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

CPU times: user 23.3 s, sys: 8.76 s, total: 32 s
Wall time: 59.3 s


<xarray.DataArray 'dem_3d' (axis: 3, y: 6250, x: 6250)>
dask.array<setitem, shape=(3, 6250, 6250), dtype=float64, chunksize=(3, 3125, 3125), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05 4e+05
    spatial_ref  int64 ...
  * y            (y) float64 4.625e+06 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
  * axis         (axis) int64 0 1 2

# acquisition

In [9]:
measurement

<xarray.DataArray 'measurement' (azimuth_time: 39406, slant_range_time: 17707)>
dask.array<open_dataset-ec63f45f09c52896b9970950c5202a86measurement, shape=(39406, 17707), dtype=complex64, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
    pixel             (slant_range_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
    line              (azimuth_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
  * azimuth_time      (azimuth_time) datetime64[ns] 2021-12-16T11:54:38.06696...
  * slant_range_time  (slant_range_time) float64 0.006087 0.006087 ... 0.006464
Attributes:
    STATISTICS_MAXIMUM:         10808
    STATISTICS_MEAN:            0.074683972276065
    STATISTICS_MINIMUM:         -10980
    STATISTICS_STDDEV:          24.790956069067
    STATISTICS_VALID_PERCENT:   100
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  4.059698
    sar:pixel_spacing_range:    3.194828
    azimuth_time_interval:      0.0006011500664149009
    slant_range_time_interval:  2.1313598509794117e-08
    sat:anx_datetime:           2021-12-16T11:16:48.149227Z
    units:                      1
    long_name:                  digital number

In [10]:
%%time
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)
acquisition

CPU times: user 1min 17s, sys: 50.8 s, total: 2min 7s
Wall time: 33.2 s


<xarray.Dataset>
Dimensions:           (x: 6250, y: 6250, axis: 3)
Coordinates:
  * x                 (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
  * axis              (axis) int64 0 1 2
Data variables:
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(3125, 3125), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(3125, 3125), meta=np.ndarray>
    dem_direction     (axis, y, x) float64 dask.array<chunksize=(3, 3125, 3125), meta=np.ndarray>

In [11]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

CPU times: user 9.23 ms, sys: 1.43 ms, total: 10.7 ms
Wall time: 10 ms


<xarray.DataArray (azimuth_time: 39406, slant_range_time: 17707)>
dask.array<pow, shape=(39406, 17707), dtype=float32, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
    pixel             (slant_range_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
    line              (azimuth_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
  * azimuth_time      (azimuth_time) datetime64[ns] 2021-12-16T11:54:38.06696...
  * slant_range_time  (slant_range_time) float64 0.006087 0.006087 ... 0.006464
Attributes:
    STATISTICS_MAXIMUM:         10808
    STATISTICS_MEAN:            0.074683972276065
    STATISTICS_MINIMUM:         -10980
    STATISTICS_STDDEV:          24.790956069067
    STATISTICS_VALID_PERCENT:   100
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  4.059698
    sar:pixel_spacing_range:    3.194828
    azimuth_time_interval:      0.0006011500664149009
    slant_range_time_interval:  2.1313598509794117e-08
    sat:anx_datetime:           2021-12-16T11:16:48.149227Z
    units:                      m2 m-2
    long_name:                  beta nought

In [12]:
%%time
coordinate_conversion = None
if measurement_ds.attrs["sar:product_type"] == "GRD":
    coordinate_conversion = xr.open_dataset(
        product_urlpath,
        engine="sentinel-1",
        group=f"{measurement_group}/coordinate_conversion",
        **kwargs,
    )  # type: ignore
    ground_range = xarray_sentinel.slant_range_time_to_ground_range(
        acquisition.azimuth_time,
        acquisition.slant_range_time,
        coordinate_conversion,
    )
    interp_kwargs = {"ground_range": ground_range}
elif measurement_ds.attrs["sar:product_type"] == "SLC":
    interp_kwargs = {"slant_range_time": acquisition.slant_range_time}
    if measurement_ds.attrs["sar:instrument_mode"] == "IW":
        beta_nought = xarray_sentinel.mosaic_slc_iw(beta_nought)
else:
    raise ValueError(
        f"unsupported sar:product_type {measurement_ds.attrs['sar:product_type']}"
    )

CPU times: user 41 µs, sys: 0 ns, total: 41 µs
Wall time: 42.9 µs


In [13]:
%%time
geocoded = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
)

geocoded

CPU times: user 9.29 s, sys: 3.37 s, total: 12.7 s
Wall time: 3.33 s


<xarray.DataArray (y: 6250, x: 6250)>
dask.array<dask_aware_interpnd, shape=(6250, 6250), dtype=float32, chunksize=(3125, 3125), chunktype=numpy.ndarray>
Coordinates:
    pixel             (y, x) float64 dask.array<chunksize=(3125, 3125), meta=np.ndarray>
    line              (y, x) float64 dask.array<chunksize=(3125, 3125), meta=np.ndarray>
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(3125, 3125), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(3125, 3125), meta=np.ndarray>
  * x                 (x) float64 3.75e+05 3.75e+05 3.75e+05 ... 4e+05 4e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.65e+06 4.65e+06
Attributes:
    STATISTICS_MAXIMUM:         10808
    STATISTICS_MEAN:            0.074683972276065
    STATISTICS_MINIMUM:         -10980
    STATISTICS_STDDEV:          24.790956069067
    STATISTICS_VALID_PERCENT:   100
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  4.059698
    sar:pixel_spacing_range:    3.194828
    azimuth_time_interval:      0.0006011500664149009
    slant_range_time_interval:  2.1313598509794117e-08
    sat:anx_datetime:           2021-12-16T11:16:48.149227Z
    units:                      m2 m-2
    long_name:                  beta nought

In [14]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [15]:
# _ = geocoded.plot(vmax=1.0)